In [1]:
from pyspark.sql import DataFrame, Row
from functools import reduce
from pyspark.sql.types import *
from pyspark.sql.functions import *
from graphframes import *
from pyspark import SparkContext
from pyspark import SparkConf
from pyspark.sql import SQLContext
import uuid

In [2]:
conf=SparkConf()
conf.set("spark.executor.memory", "2g")
conf.set("spark.driver.memory", "12g")
conf.set("spark.cores.max", "6")

sc = SparkContext.getOrCreate(conf)

spark = SQLContext(sc)

23/12/06 09:53:24 WARN Utils: Your hostname, preacher resolves to a loopback address: 127.0.1.1; using 192.168.1.6 instead (on interface enp4s0)
23/12/06 09:53:24 WARN Utils: Set SPARK_LOCAL_IP if you need to bind to another address
Using Spark's default log4j profile: org/apache/spark/log4j-defaults.properties
Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
23/12/06 09:53:25 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable
/home/preacher/anaconda3/envs/deltarun/lib/python3.9/site-packages/pyspark/sql/context.py:77: FutureWarning: Deprecated in 3.0.0. Use SparkSession.builder.getOrCreate() instead.
  warnings.warn(


In [3]:
match_schema = StructType([
    StructField("pair_id", StringType(), True),
    StructField("uuid_a", StringType(), True),
    StructField("uuid_b", StringType(), True),
    StructField("score", DoubleType(), True),
    StructField("segment", StringType(), True),
    StructField("seller_flag", StringType(), True),
    StructField("year", StringType(), True),
    StructField("month", StringType(), True),
    StructField("day", StringType(), True),
    StructField("run_number", StringType(), True),
    StructField("source", StringType(), True),
])


In [4]:
upc_match = [
    ["0000346280418<>203359001<>homedepot<>homedepot_346280418<>g4103325<>bcg<>bcg", "0000346280418<>203359001<>homedepot<>homedepot","346280418<>g4103325<>bcg<>bcg",1.0,"pets","1P","2023","07","17", "01","UPC"],
    ["0000346318654<>100341666<>homedepot<>homedepot_000346318654<>109589<>hdsupplysolutions<>hdsupplysolutions", "0000346318654<>100341666<>homedepot<>homedepot","000346318654<>109589<>hdsupplysolutions<>hdsupplysolutions",1.0,"pets","1P","2023","07","17", "01","UPC"],
    ["0000346378252<>203370169<>homedepot<>homedepot_000346378252<>967446<>hdsupplysolutions<>hdsupplysolutions", "0000346378252<>203370169<>homedepot<>homedepot","000346378252<>967446<>hdsupplysolutions<>hdsupplysolutions",1.0,"pets","1P","2023","07","17", "01","UPC"],
    ["0000346383867<>100670057<>homedepot<>homedepot_000346383867<>967493<>hdsupplysolutions<>hdsupplysolutions", "0000346383867<>100670057<>homedepot<>homedepot","000346383867<>967493<>hdsupplysolutions<>hdsupplysolutions",1.0,"pets","1P","2023","07","17", "01","UPC"],
    ["00003463186541<>100341666<>homedepot<>homedepot_000346318654<>109589<>hdsupplysolutions<>hdsupplysolutions", "00003463186541<>100341666<>homedepot<>homedepot","000346318654<>109589<>hdsupplysolutions<>hdsupplysolutions",1.0,"pets","1P","2023","07","17", "01","UPC"], # > 0.05
    ["0000000000001<>53409<>petcarerx<>petcarerx_00000432<>147664<>chewy<>chewy", "0000000000001<>53409<>petcarerx<>petcarerx","00000432<>147664<>chewy<>chewy",1.0,"pets","1P","2023","07","17", "01","UPC"], # appended one old not present
    ["0000000000001<>53409<>petcarerx<>petcarerx_00000000<>147664<>chewy<>chewy", "0000000000001<>53409<>petcarerx<>petcarerx","00000000<>147664<>chewy<>chewy",1.0,"pets","1P","2023","07","17", "01","UPC"], # appended
]

In [5]:
mpn_match = [
    ["0000346280418<>203359001<>homedepot<>homedepot_346280418<>g4103325<>bcg<>bcg","0000346280418<>203359001<>homedepot<>homedepot","346280418<>g4103325<>bcg<>bcg",1.0,"pets","1P","2023","07","17", "01","MPN"],
    ["0000346318654<>100341666<>homedepot<>homedepot_000346318654<>109589<>hdsupplysolutions<>hdsupplysolutions","0000346318654<>100341666<>homedepot<>homedepot","000346318654<>109589<>hdsupplysolutions<>hdsupplysolutions",1.0,"pets","1P","2023","07","17", "01","MPN"],
    ["0000346383843<>202080350<>homedepot<>homedepot_346383843<>g1969694<>bcg<>bcg","0000346383843<>202080350<>homedepot<>homedepot","346383843<>g1969694<>bcg<>bcg",1.0,"pets","1P","2023","07","17", "01","MPN"],
    ["0000346392418<>204823270<>homedepot<>homedepot_000346392418<>453560<>hdsupplysolutions<>hdsupplysolutions","0000346392418<>204823270<>homedepot<>homedepot","000346392418<>453560<>hdsupplysolutions<>hdsupplysolutions",1.0,"pets","1P","2023","07","17", "01","MPN"],
    ["00003463186541<>100341666<>homedepot<>homedepot_0003463186541<>109589<>hdsupplysolutions<>hdsupplysolutions","00003463186541<>100341666<>homedepot<>homedepot","0003463186541<>109589<>hdsupplysolutions<>hdsupplysolutions",1.0,"pets","1P","2023","07","17", "01","MPN"], # > 0.05 + added
    ["00003463782521<>203370169<>homedepot<>homedepot_000346378252<>967446<>hdsupplysolutions<>hdsupplysolutions","00003463782521<>203370169<>homedepot<>homedepot","000346378252<>967446<>hdsupplysolutions<>hdsupplysolutions",1.0,"pets","1P","2023","07","17", "01","MPN"], # < 0.05
    ["00003463782521<>203370169<>homedepot<>homedepot_000346378252<>9674462<>hdsupplysolutions<>hdsupplysolutions","00003463782521<>203370169<>homedepot<>homedepot","000346378252<>9674462<>hdsupplysolutions<>hdsupplysolutions",1.0,"pets","1P","2023","07","17", "01","MPN"], # < 0.05 + added
    ["0000000000001<>53409<>petcarerx<>petcarerx_00000000<>147664<>chewy<>chewy","0000000000001<>53409<>petcarerx<>petcarerx","00000000<>147664<>chewy<>chewy",1.0,"pets","1P","2023","07","17", "01","MPN"], # appended
]

In [6]:
ml_data = [
    ["0000346280418<>203359001<>homedepot<>homedepot_346280418<>g4103325<>bcg<>bcg","0000346280418<>203359001<>homedepot<>homedepot","346280418<>g4103325<>bcg<>bcg",0.987,"pets","1P","2023","07","17", "01","ML"],
    ["0000346378252<>203370169<>homedepot<>homedepot_000346378252<>967446<>hdsupplysolutions<>hdsupplysolutions","0000346378252<>203370169<>homedepot<>homedepot","000346378252<>967446<>hdsupplysolutions<>hdsupplysolutions",0.876,"pets","1P","2023","07","17", "01","ML"],
    ["0000346383843<>202080350<>homedepot<>homedepot_346383843<>g1969694<>bcg<>bcg","0000346383843<>202080350<>homedepot<>homedepot","346383843<>g1969694<>bcg<>bcg",0.567,"pets","1P","2023","07","17", "01","ML"],
    ["000000000000<>53409<>petcarerx<>petcarerx_000000000000<>53417<>petcarerx<>petcarerx","000000000000<>53409<>petcarerx<>petcarerx","000000000000<>53417<>petcarerx<>petcarerx",0.8737,"pets","1P","2023","07","17", "01","ML"],
    ["00003463186541<>100341666<>homedepot<>homedepot_000346318654<>109589<>hdsupplysolutions<>hdsupplysolutions","00003463186541<>100341666<>homedepot<>homedepot","000346318654<>109589<>hdsupplysolutions<>hdsupplysolutions",0.8737,"pets","1P","2023","07","17", "01","ML"], # > 0.05
    ["00003463186541<>100341666<>homedepot<>homedepot_0003463186541<>109589<>hdsupplysolutions<>hdsupplysolutions","00003463186541<>100341666<>homedepot<>homedepot","0003463186541<>109589<>hdsupplysolutions<>hdsupplysolutions",0.8737,"pets","1P","2023","07","17", "01","ML"], # > 0.05 + added
    ["00003463782521<>203370169<>homedepot<>homedepot_000346378252<>967446<>hdsupplysolutions<>hdsupplysolutions","00003463782521<>203370169<>homedepot<>homedepot","000346378252<>967446<>hdsupplysolutions<>hdsupplysolutions",0.943,"pets","1P","2023","07","17", "01","ML"], # < 0.05
    ["00003463782521<>203370169<>homedepot<>homedepot_000346378252<>9674462<>hdsupplysolutions<>hdsupplysolutions","00003463782521<>203370169<>homedepot<>homedepot","000346378252<>9674462<>hdsupplysolutions<>hdsupplysolutions",0.943,"pets","1P","2023","07","17", "01","ML"], # < 0.05 + added
]

In [7]:
transitive_data = [
    ["0000346318654<>100341666<>homedepot<>homedepot_000346318654<>109589<>hdsupplysolutions<>hdsupplysolutions", "0000346318654<>100341666<>homedepot<>homedepot","000346318654<>109589<>hdsupplysolutions<>hdsupplysolutions",1.0,"pets","1P","2023","07","17", "01","TRANSITIVE"],
    ["0000346378252<>203370169<>homedepot<>homedepot_000346378252<>967446<>hdsupplysolutions<>hdsupplysolutions", "0000346378252<>203370169<>homedepot<>homedepot","000346378252<>967446<>hdsupplysolutions<>hdsupplysolutions",1.0,"pets","1P","2023","07","17", "01","TRANSITIVE"],
    ["0000346383843<>202080350<>homedepot<>homedepot_346383843<>g1969694<>bcg<>bcg", "0000346383843<>202080350<>homedepot<>homedepot","346383843<>g1969694<>bcg<>bcg",1.0,"pets","1P","2023","07","17", "01","TRANSITIVE"],
    ["000000000000<>53409<>petcarerx<>petcarerx_00000000<>147664<>chewy<>chewy", "000000000000<>53409<>petcarerx<>petcarerx","00000000<>147664<>chewy<>chewy",1.0,"pets","1P","2023","07","17", "01","TRANSITIVE"],
    ["00003463782521<>203370169<>homedepot<>homedepot_000346378252<>9674462<>hdsupplysolutions<>hdsupplysolutions", "00003463782521<>203370169<>homedepot<>homedepot","000346378252<>9674462<>hdsupplysolutions<>hdsupplysolutions",1.0,"pets","1P","2023","07","17", "01","TRANSITIVE"], # < 0.05 + added
    ["0000000000001<>53409<>petcarerx<>petcarerx_00000123<>147664<>chewy<>chewy", "0000000000001<>53409<>petcarerx<>petcarerx","00000123<>147664<>chewy<>chewy",1.0,"pets","1P","2023","07","17", "01","TRANSITIVE"], # added
    ["0000000000001<>53409<>petcarerx<>petcarerx_00000432<>147664<>chewy<>chewy", "0000000000001<>53409<>petcarerx<>petcarerx","00000432<>147664<>chewy<>chewy",1.0,"pets","1P","2023","07","17", "01","TRANSITIVE"], # appended one old not present
    ["0000000000001<>53409<>petcarerx<>petcarerx_00000000<>147664<>chewy<>chewy", "0000000000001<>53409<>petcarerx<>petcarerx","00000000<>147664<>chewy<>chewy",1.0,"pets","1P","2023","07","17", "01","TRANSITIVE"], # appended
]

In [8]:
mdw_schema = StructType([
    StructField("pair_id", StringType(), True),
    StructField("base_sku_uuid", StringType(), True),
    StructField("comp_sku_uuid", StringType(), True),
    StructField("segment", StringType(), True),
    StructField("match_source", MapType(StringType(), DoubleType()), True),
    StructField("score", DoubleType(), True),
])

In [9]:
mdw_data = [
    ["00003463186541<>100341666<>homedepot<>homedepot_000346318654<>109589<>hdsupplysolutions<>hdsupplysolutions", "00003463186541<>100341666<>homedepot<>homedepot","000346318654<>109589<>hdsupplysolutions<>hdsupplysolutions","pets",{"ML": 0.745, "UPC": 1.0},1.745], # > 0.05
    ["00003463186541<>100341666<>homedepot<>homedepot_0003463186541<>109589<>hdsupplysolutions<>hdsupplysolutions", "00003463186541<>100341666<>homedepot<>homedepot","0003463186541<>109589<>hdsupplysolutions<>hdsupplysolutions","pets",{"ML": 0.745, "MPN": 1.0},1.745], # > 0.05 + added
    ["00003463782521<>203370169<>homedepot<>homedepot_000346378252<>967446<>hdsupplysolutions<>hdsupplysolutions", "00003463782521<>203370169<>homedepot<>homedepot","000346378252<>967446<>hdsupplysolutions<>hdsupplysolutions","pets",{"ML": 0.945, "MPN": 1.0},0.945], # < 0.05
    ["00003463782521<>203370169<>homedepot<>homedepot_000346378252<>9674462<>hdsupplysolutions<>hdsupplysolutions", "00003463782521<>203370169<>homedepot<>homedepot","000346378252<>9674462<>hdsupplysolutions<>hdsupplysolutions","pets",{"ML": 0.945, "MPN": 1.0},1.945], # < 0.05 + added
    ["00003463838431<>202080350<>homedepot<>homedepot_346383843<>g1969694<>bcg<>bcg", "00003463838431<>202080350<>homedepot<>homedepot","346383843<>g1969694<>bcg<>bcg","pets", {"UPC": 1.0, "MPN": 1.0},2.0], # not added
    ["0000000000001<>53409<>petcarerx<>petcarerx_00000000<>147664<>chewy<>chewy", "0000000000001<>53409<>petcarerx<>petcarerx","00000000<>147664<>chewy<>chewy","pets", {"UPC": 1.0, "MPN": 1.0},2.0], # appended
    ["0000000000001<>53409<>petcarerx<>petcarerx_00000432<>147664<>chewy<>chewy", "0000000000001<>53409<>petcarerx<>petcarerx","00000432<>147664<>chewy<>chewy","pets", {"UPC": 1.0, "MPN": 1.0},2.0], # appended one old not present
    ["0000000000001<>53409<>petcarerx<>petcarerx_00000123<>147664<>chewy<>chewy", "0000000000001<>53409<>petcarerx<>petcarerx","00000123<>147664<>chewy<>chewy","pets", {"UPC": 1.0, "MPN": 1.0},2.0], # added
]

In [10]:
mdw = spark.createDataFrame(mdw_data, schema=mdw_schema)

In [11]:
ml_match_suggestion = df = spark.createDataFrame(ml_data, schema=match_schema)
upc_match_suggestion = spark.createDataFrame(upc_match, schema=match_schema)
mpn_match_suggestion = spark.createDataFrame(mpn_match, schema=match_schema)
transitive_match_suggestion = spark.createDataFrame(transitive_data, schema=match_schema)


In [12]:
from aggregator import Aggregator
aggregator = Aggregator(ml_match_suggestion,
upc_match_suggestion,
mpn_match_suggestion,
transitive_match_suggestion,
mdw)

Combining dfs


In [13]:
merged_matches = aggregator.merged_match_suggestion
mdw = aggregator.mdw

In [14]:
merged_matches.printSchema()
mdw.printSchema()

root
 |-- pair_id: string (nullable = true)
 |-- uuid_a: string (nullable = true)
 |-- uuid_b: string (nullable = true)
 |-- new_score: double (nullable = true)
 |-- new_segment: string (nullable = true)
 |-- new_seller_flag: string (nullable = true)
 |-- year: string (nullable = true)
 |-- month: string (nullable = true)
 |-- day: string (nullable = true)
 |-- run_number: string (nullable = true)
 |-- source: string (nullable = true)

root
 |-- pair_id: string (nullable = true)
 |-- base_sku_uuid: string (nullable = true)
 |-- comp_sku_uuid: string (nullable = true)
 |-- segment: string (nullable = true)
 |-- match_source: map (nullable = true)
 |    |-- key: string
 |    |-- value: double (valueContainsNull = true)
 |-- score: double (nullable = true)



In [15]:
merged_matches = aggregator.aggregate_match_suggestion_and_mdw(mdw, merged_matches)

+--------------------+--------------------+--------------------+-----------+---------+------+-----+
|             pair_id|       base_sku_uuid|       comp_sku_uuid|old_segment|old_score|source|score|
+--------------------+--------------------+--------------------+-----------+---------+------+-----+
|00003463186541<>1...|00003463186541<>1...|000346318654<>109...|       pets|    1.745|   UPC|  1.0|
|00003463186541<>1...|00003463186541<>1...|000346318654<>109...|       pets|    1.745|    ML|0.745|
|00003463186541<>1...|00003463186541<>1...|0003463186541<>10...|       pets|    1.745|   MPN|  1.0|
|00003463186541<>1...|00003463186541<>1...|0003463186541<>10...|       pets|    1.745|    ML|0.745|
|00003463782521<>2...|00003463782521<>2...|000346378252<>967...|       pets|    0.945|   MPN|  1.0|
|00003463782521<>2...|00003463782521<>2...|000346378252<>967...|       pets|    0.945|    ML|0.945|
|00003463782521<>2...|00003463782521<>2...|000346378252<>967...|       pets|    1.945|   MPN|  1.0|


AnalysisException: cannot resolve 'old_seller_flag' given input columns: [base_sku_uuid, comp_sku_uuid, day, month, new_score, new_seller_flag, old_score, pair_id, run_number, score, segment, source, uuid_a, uuid_b, year];
'Project [pair_id#234, source#235, uuid_a#13, uuid_b#14, new_score#166, new_seller_flag#168, year#18, month#19, day#20, run_number#21, base_sku_uuid#1, comp_sku_uuid#2, old_score#181, score#351, segment#368, CASE WHEN isnull('old_seller_flag) THEN new_seller_flag#168 ELSE 'old_seller_flag END AS seller_flag#401]
+- Project [pair_id#234, source#235, uuid_a#13, uuid_b#14, new_score#166, new_seller_flag#168, year#18, month#19, day#20, run_number#21, base_sku_uuid#1, comp_sku_uuid#2, old_score#181, score#351, segment#368]
   +- Project [pair_id#234, source#235, uuid_a#13, uuid_b#14, new_score#166, new_segment#167, new_seller_flag#168, year#18, month#19, day#20, run_number#21, base_sku_uuid#1, comp_sku_uuid#2, old_segment#180, old_score#181, score#351, CASE WHEN isnull(ol

In [ ]:
merged_matches.show()

+--------------------+----------+--------------------+--------------------+---------+-----------+-----------+----+-----+----+----------+--------------------+--------------------+-----------+---------+------+-------+------------+
|             pair_id|    source|              uuid_a|              uuid_b|new_score|new_segment|seller_flag|year|month| day|run_number|       base_sku_uuid|       comp_sku_uuid|old_segment|old_score| score|segment|match_status|
+--------------------+----------+--------------------+--------------------+---------+-----------+-----------+----+-----+----+----------+--------------------+--------------------+-----------+---------+------+-------+------------+
|0000000000001<>53...|       MPN|0000000000001<>53...|00000000<>147664<...|      1.0|       pets|         1P|2023|   07|  17|        01|0000000000001<>53...|00000000<>147664<...|       pets|      2.0|   1.0|   pets|     updated|
|0000000000001<>53...|TRANSITIVE|0000000000001<>53...|00000000<>147664<...|      1.0

In [ ]:
updated_matches_suggestion, new_matches_suggestion = aggregator.fetch_updated_and_new_matches_suggestions(merged_matches)

AnalysisException: cannot resolve 'base_source_store' given input columns: [base_sku_uuid, comp_sku_uuid, day, match_status, month, new_score, new_segment, old_score, old_segment, pair_id, run_number, score, segment, seller_flag, source, uuid_a, uuid_b, year];
'Aggregate [pair_id#233, segment#364, base_sku_uuid#1, comp_sku_uuid#2, 'base_source_store, 'comp_source_store, 'seller_type], [pair_id#233, segment#364, base_sku_uuid#1, comp_sku_uuid#2, 'base_source_store, 'comp_source_store, 'seller_type, map_from_entries(collect_list(struct(source, source#234, score, score#347), 0, 0)) AS match_source#943, collect_set(match_status#382, 0, 0) AS match_status#945, sum(score#347) AS score#947, collect_set('match_status_col_name, 0, 0) AS match_status_col_name#949]
+- Project [pair_id#233, source#234, uuid_a#13, uuid_b#14, new_score#166, new_segment#167, seller_flag#17, year#18, month#19, day#20, run_number#21, base_sku_uuid#1, comp_sku_uuid#2, old_segment#179, old_score#180, score#347, segment#3